In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from coderm.model import EvolverModel

In [3]:
model = EvolverModel("/mnt/efs/federicocassano/t_models/generator-sc2-15b", "/mnt/efs/federicocassano/codeprm/training/code-scorer/15b_orm/", evolver_e=3)

INFO 06-24 18:36:16 config.py:1218] Downcasting torch.float32 to torch.bfloat16.
INFO 06-24 18:36:16 llm_engine.py:161] Initializing an LLM engine (v0.5.0.post1) with config: model='/mnt/efs/federicocassano/t_models/generator-sc2-15b', speculative_config=None, tokenizer='/mnt/efs/federicocassano/t_models/generator-sc2-15b', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=16384, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=/mnt/efs/federicocassano/t_models/generator-sc2-15b)
INFO 06-24 18:36:17 selector.py:150] Cannot use FlashAttention-2 backend due to sliding window.
INFO 06-24 18:36:17 selecto

The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [4]:
import datasets
ds = datasets.load_dataset("codegenning/livecodebench_lite_filtered", split="test")
ds

Dataset({
    features: ['question', 'starter_code', 'difficulty', 'input_output', 'public_input_output', 'title', 'source', 'date', 'id'],
    num_rows: 202
})

In [5]:
q = ds[0]["question"]
p = model.format_prompt(q)
print(p)

"""
Vanya and Vova are playing a game. Players are given an integer $n$. On their turn, the player can add $1$ to the current integer or subtract $1$. The players take turns; Vanya starts. If after Vanya's move the integer is divisible by $3$, then he wins. If $10$ moves have passed and Vanya has not won, then Vova wins.

Write a program that, based on the integer $n$, determines who will win if both players play optimally.

Input

The first line contains the integer $t$ ($1 \leq t \leq 100$) — the number of test cases.

The single line of each test case contains the integer $n$ ($1 \leq n \leq 1000$).

Output

For each test case, print "First" without quotes if Vanya wins, and "Second" without quotes if Vova wins.Sample Input 1:
6

1

3

5

100

999

1000



Sample Output 1:

First
Second
First
First
Second
First

"""


In [18]:
c = model.generate_with_info([p], temperature=0.4)[0]
print(c.orm_score)
print(c.code)

Evolver iterations:   0%|                                                                                                                                                                       | 0/3 [00:00<?, ?it/s]

[]
Evolved prompts
"""
Vanya and Vova are playing a game. Players are given an integer $n$. On their turn, the player can add $1$ to the current integer or subtract $1$. The players take turns; Vanya starts. If after Vanya's move the integer is divisible by $3$, then he wins. If $10$ moves have passed and Vanya has not won, then Vova wins.

Write a program that, based on the integer $n$, determines who will win if both players play optimally.

Input

The first line contains the integer $t$ ($1 \leq t \leq 100$) — the number of test cases.

The single line of each test case contains the integer $n$ ($1 \leq n \leq 1000$).

Output

For each test case, print "First" without quotes if Vanya wins, and "Second" without quotes if Vova wins.Sample Input 1:
6

1

3

5

100

999

1000



Sample Output 1:

First
Second
First
First
Second
First

"""


Evolver iterations:  33%|█████████████████████████████████████████████████████                                                                                                          | 1/3 [00:02<00:04,  2.13s/it]



# Read the number of test cases
t = int(input())

# Iterate through each test case
for i in range(t):
    # Read the integer n
    n = int(input())

    # Check if Vova will win
    if (n + 2) % 3 == 0:
        # If Vova will win, print "Second"
        print('Second')
    else:
        # If Vova will not win, print "First"
        print('First')
0.0255126953125
Evolved prompts
"""
Vanya and Vova are playing a game. Players are given an integer $n$. On their turn, the player can add $1$ to the current integer or subtract $1$. The players take turns; Vanya starts. If after Vanya's move the integer is divisible by $3$, then he wins. If $10$ moves have passed and Vanya has not won, then Vova wins.

Write a program that, based on the integer $n$, determines who will win if both players play optimally.

Input

The first line contains the integer $t$ ($1 \leq t \leq 100$) — the number of test cases.

The single line of each test case contains the integer $n$ ($1 \leq n \leq 1000$).

Output

Evolver iterations:  67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████                                                     | 2/3 [00:06<00:03,  3.40s/it]


# Read the number of test cases
t = int(input())

# Iterate through each test case
for i in range(t):
    # Read the integer n
    n = int(input())

    # Check if Vova will win
    if n % 3 == 0:
        # If Vova will win, print "Second"
        print('Second')
    elif n % 3 == 2:
        # If Vova will win by adding 1, print "First"
        print('First')
    elif n % 3 == 1:
        # If Vova will win by subtracting 1, check if the number of moves is less than or equal to 10
        if (n - 1) // 3 <= 10:
            # If the number of moves is less than or equal to 10, print "First"
            print('First')
        else:
            # If the number of moves is greater than 10, print "Second"
            print('Second')
0.006500244140625
[<coderm.model.Completion object at 0x7f3140ad4d90>, <coderm.model.Completion object at 0x7f3140ad72e0>]
Evolved prompts
"""
Vanya and Vova are playing a game. Players are given an integer $n$. On their turn, the player can add $1$ to the curre

Evolver iterations: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:09<00:00,  3.23s/it]



# Read the number of test cases
t = int(input())

# Iterate through each test case
for i in range(t):
    # Read the integer n
    n = int(input())

    # Check if Vova will win
    if (n - 1) % 3 == 0:
        # If Vova will win, print "Second"
        print('Second')
    elif n % 3 == 0:
        # If Vova will win, print "Second"
        print('Second')
    elif (n + 1) % 3 == 0:
        # If Vova will win, print "Second"
        print('Second')
    else:
        # If Vova will not win, print "First"
        print('First')
0.70703125
0.70703125


# Read the number of test cases
t = int(input())

# Iterate through each test case
for i in range(t):
    # Read the integer n
    n = int(input())

    # Check if Vova will win
    if (n - 1) % 3 == 0:
        # If Vova will win, print "Second"
        print('Second')
    elif n % 3 == 0:
        # If Vova will win, print "Second"
        print('Second')
    elif (n + 1) % 3 == 0:
        # If Vova will win, print "Second"
        print('